In [1]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sergi\PycharmProjects\PythonEnv\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [134]:
from pymongo import MongoClient
import json

# Подключение к MongoDB
client = MongoClient('localhost', 27017)


client.drop_database("University")  # Удаление базы данных если существует уже

db = client['University']

# Функция для вставки ответа в коллекцию Answers
def insert_to_answer(id, ans):
    db.Answers.update_one(
        {"_id": id},
        {"$set": {"answer": ans}},
        upsert=True
    )
    return ans


# 1. Импортируйте Students.json в локальный MongoDB, DB University, Collection Students.
# Ключ задайте по полю Номер.
students = db['Students']  # создание коллекции Students


def load_students_data():
    '''
    Функция загрузки данных в коллекцию
    '''
    with open("Students.json", encoding="utf-8-sig") as f:
        students_json = json.load(f)
        for student in students_json:
            try:
                student['_id'] = student['Номер']
            except:
                print(f'ошибка назначения _id при {student}')
        students.insert_many(students_json)


load_students_data()

# 2. Выведите статистику по DB. Какое общее количество объектов?
insert_to_answer(2, db.command('dbstats')['objects'])

# 3. Какое общее количество документов в коллекции Students?
insert_to_answer(3, db.Students.count_documents({}))

# 4. Сколько студентов обучается на специальности Математика?
insert_to_answer(4, db.Students.count_documents(
    {"Специальность": "Математика"}))

# 5. Сколько студентов обучается на специальности Математика на втором курсе?
insert_to_answer(5, db.Students.count_documents(
    {"Специальность": "Математика", "Курс": 2}))

# 6. Сколько студентов обучается на специальности Математика кроме второго курса?
insert_to_answer(6, db.Students.count_documents(
    {"Специальность": "Математика", "Курс": {"$ne": 2}}))

# 7. Сколько студентов обучается на специальности Математика старше третьего курса?
insert_to_answer(7, db.Students.count_documents(
    {"Специальность": "Математика", "Курс": {"$gt": 3}}))

# 8. Сколько студентов обучается на специальности Юриспруденция и Экономика кроме третьего курса?
insert_to_answer(8, db.Students.count_documents({"$or": [
                 {"Специальность": "Юриспруденция"}, {"Специальность": "Экономика"}], "Курс": {"$ne": 3}}))

# 9. Граждане скольких стран обучаются в ВУЗе?
insert_to_answer(9, len(db.Students.distinct("Гражданство")))

# 10. Вывести в файл 10 первых документов из коллекции Students только с полями _id, Факультет, Гражданство. Отсортировать по полю Гражданство.
result = db.Students.find({}, {"_id": 1, "Факультет": 1, "Гражданство": 1}).sort(
    "Гражданство", 1).limit(10)
insert_to_answer(10, list(result))

# 11. Импорт Languages2.json в коллекцию Languages2
with open("Languages2.json", encoding="utf-8-sig") as f:
    languages_json = json.load(f)
db.Languages2.insert_many(languages_json)

# Создание объекта view для коллекций Students и Languages2 с JOIN-ом
pipeline = [
    {
        "$lookup": {
            "from": "Languages2",
            "localField": "idLanguage",
            "foreignField": "_id",
            "as": "languageInfo"
        }
    },
    {
        "$unwind": "$languageInfo"
    },
    {
        "$addFields": {"Языки": "$languageInfo.Languages"}
    },
    {
        "$project": {
            "languageInfo": 0,
            "idLanguage": 0
        }
    }
]
db.create_collection("StudentsLanguages", viewOn="Students", pipeline=pipeline)

# 12. Какие языки изучает студент 1032102469?
result = db.StudentsLanguages.find(
    {"Номер": "1032102469"}, {"Языки": 1, "_id": 0})
insert_to_answer(12, result[0]["Языки"])

# 13. Сколько студентов изучают какие-либо языки?
insert_to_answer(13, db.Students.count_documents(
    {"idLanguage": {"$exists": True, "$ne": []}}))

# 14. Сколько студентов не изучают никаких языков?
insert_to_answer(14,  db.Students.count_documents({ "idLanguage": { "$exists": False } }))

# 15. Сколько студентов изучают немецкий язык?
insert_to_answer(
    15, db.StudentsLanguages.count_documents({"Языки": "немецкий"}))

# 16. Какой _id у документа в коллекции Languages2, содержащего массив из трех языков?
result = db.Languages2.find({"Languages": {"$size": 3}}, {"_id": 1})
insert_to_answer(16, result[0]["_id"])

# 17. Сколько студентов изучают три языка?
insert_to_answer(17, db.StudentsLanguages.count_documents(
    {"Языки": {"$size": 3}}))

# 18. Сколько студентов изучают пять языков?
insert_to_answer(18, db.Languages2.count_documents(
    {"Languages": {"$size": 5}}))

# 19. Создайте индексы в коллекции Students для нескольких полей и один составной индекс.
db.Students.create_index([("Факультет", 1)])
db.Students.create_index([("Специальность", 1)])
db.Students.create_index([("Курс", 1)])
db.Students.create_index([("Факультет", 1), ("Специальность", 1), ("Курс", 1)])

# 20. Увеличьте всем студентам курс на 1. Максимальный курс может быть 6.
db.Students.update_many({"Курс": {"$lt": 6}}, {"$inc": {"Курс": 1}})
result = db.Students.find().skip(499).limit(2000)
insert_to_answer(20, list(result))

# 21. Сколько студентов поступило в 2009 году?
insert_to_answer(21, db.Students.count_documents(
    {"Номер": {"$regex": "^.{3}209.*$"}}))

# 22. Сколько студентов поступило с 2000 по 2005 годы?
insert_to_answer(22, db.Students.count_documents(
    {"Номер": {"$regex": "^.{3}20[0-5].*$"}}))

# 23. Замените в файле Students.json несколько значений Курс с числового на текстовый и загрузите в новую коллекцию Students2
# У первых 5 записей меняем тип на строковый
db.Students.aggregate([{"$match": {}}, {"$out": "Students2"}])
for doc in list(db['Students'].find({}).limit(5)):
    db['Students2'].update_one(
    { "_id": doc['_id'] },
    { "$set": { "Курс": str(doc['Курс']) } })

# Вычислите количество значений Курс числовое и текстовое
result = list(db.Students2.aggregate([
    {"$group": {"_id": {"$type": "$Курс"}, "count": {"$sum": 1}}},
    {"$project": {"type": "$_id", "count": "$count", "_id": 0}}
]))
insert_to_answer(23, result)

# 24. Добавьте в коллекцию Students каждому студенту соответствующий массив Languages. Выведите в файл с 100 по 200 записи
result = db.StudentsLanguages.find().skip(99).limit(200)
insert_to_answer(24, list(result))


from bson import json_util
answers_data = list(db.Answers.find())

# Экспорт данных в JSON файл
with open('answers-python.json', 'w', encoding='utf-8') as json_file:
    json.dump(answers_data, json_file, default=json_util.default, ensure_ascii=False, indent=2)

# Закрываем соединение с MongoDB
client.close()

ошибка назначения _id при {}


In [18]:
# from bson import json_util

# # Подключение к MongoDB
# client = MongoClient('mongodb://localhost:27017/')
# db = client['University']


# # Запрос для получения данных из коллекции Answers
# answers_data = list(db.Answers.find())

# # Экспорт данных в JSON файл
# with open('answers_export.json', 'w', encoding='utf-8') as json_file:
#     json.dump(answers_data, json_file, default=json_util.default, ensure_ascii=False, indent=2)

# # # Закрываем соединение с MongoDB
# client.close()

In [136]:
file1_path = "./answers-js.json"
file2_path = "./answers-python.json"


with open(file1_path, 'r', encoding="utf-8-sig") as file1:
    data1 = json.load(file1)

with open(file2_path, 'r', encoding="utf-8-sig") as file2:
    data2 = json.load(file2)

In [137]:
data1_dict = {doc['_id']:doc['answer'] for doc in data1}
data2_dict = {doc['_id']:doc['answer'] for doc in data2}

In [138]:
for k in data1_dict:
    
    if data1_dict[k] != data2_dict[k]:
        print(k)
        print(data1_dict[k])
        print(data2_dict[k])

10
[{'_id': '6568be35c58f3796981faad2'}, {'_id': '1032105325', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Инженерный'}, {'_id': '1032055006', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Экономический'}, {'_id': '1032095065', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Филологический'}, {'_id': '1032125215', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Филологический'}, {'_id': '1032135063', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Институт иностранных языков'}, {'_id': '1032145164', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Физико-математических  и естественных наук'}, {'_id': '1032105324', 'Гражданство': '(КНДР) Корейская Народно - Демократическая Республика', 'Факультет': 'Филологический'}, {'_id': '1032095152', 'Гражданство': '(КН

In [139]:
data1_dict[10][1:] == data2_dict[10][1:] # В 10 тоже совпадают ответы

True